# Data Loader

> Data Loader Object for the EEG data

In [3]:
#| default_exp data_loader

In [4]:
#| hide
from nbdev.showdoc import *

In [5]:
#| export
import numpy as np
import h5py
from scipy.signal import firwin, filtfilt
from scipy.signal import resample

In [103]:
#| export
class DataLoader:
    """Load and Preprocess the EEG Raw Data"""

    def __init__(self,
                 all_data_fp="", # the all data file's path
                 ):
        """Initialize DataLoader with the data file path"""
        # data related fields
        self.all_data_fp = all_data_fp
        self.all_data = None
        self.source_info = None
        self.resp_info = None
        self.num_participant = None
        # h5py data reference. Will be stored in the format of
        # (#participant, source_info) : <HDF5 object reference>
        self.data_refs = dict()
        if self.all_data_fp != "":
            self.load_data_ref(all_data_fp)


    def load_data_ref(self,
                     all_data_fp:str, # all data file's path
                     all_data_key = "ALLDATA", # the MatLab variable that keeps track of all raw data
                    ):
        """Load the data reference with h5py module, without laoding into RAM"""
        # load the dataset
        self.all_data = h5py.File(all_data_fp, 'r')[all_data_key]
        # extract useful dataset-related attributes
        self.source_info = list(self.all_data.keys())
        self.num_participant = self.all_data[self.source_info[0]].shape[0]
        # iterate through all participants in
        for participant in range(self.num_participant):
            for source in self.source_info:
                key = (participant, source)
                ref = self.all_data[source][participant][0]
                self.data_refs[key] = ref

    def _check_init(self):
        if self.all_data is None:
            raise ValueError("All Data has not been loaded. Please load via load_data_ref")

    def _get_datum(self,
                  participant:int, # the participant number
                  source: str, # source information, one of ['CR', 'FA', 'IR', 'SC', 'SI']
                  ) -> h5py._hl.dataset.Dataset:
        self._check_init()
        return self.all_data[self.data_refs[(participant, source)]]["data"]

    def get_data(self,
                 participant: int, # the associate data corresponding to this participant
                 ):
        self._check_init()
        data = []
        for source in self.source_info:
            dataset = np.array(self._get_datum(participant=participant, source=source))
            data.append(dataset)
        return np.concatenate(data, axis=0)

    def preprocess_by_trial(self,
                            participant: int, # the participant number that is preprocessing with
                            reref=None,
                            reref_t=None,
                            freq_bands=None,
                            f_order=None,
                            downrate=None,
                            t_interval=None
                            ):
        # get params from structure
        input_struct = self.data_refs[(participant, self.source_info[0])]
        xmin = self.all_data[input_struct]['xmin'][0]
        xmax = self.all_data[input_struct]['xmax'][0]
        srate = self.all_data[input_struct]['srate'][0]
        input_data = self.get_data(participant)

        # get params from inputs
        if reref is None:
            reref = 0

        if reref_t is None:
            reref_t = abs(xmin)

        if freq_bands is None:
            freq_bands = 0

        if downrate is None:
            p = 1
            q = 1
        elif len(downrate) == 2:
            p = downrate[0]
            q = downrate[1]
        else:
            print('error in downrate')
            return

        if t_interval is None:
            start_t = xmin
            end_t = xmax
            start_s = 0
            end_s = len(self.all_data[input_struct]['times'][0])
        elif len(t_interval) == 2:
            start_t = t_interval[0]
            end_t = t_interval[1]
            start_s = int((start_t - xmin) * srate)
            end_s = int((end_t - xmin) * srate)
        else:
            print('error in t_interval')
            return

        # Preprocessing
        num_of_channels, seq_length, num_of_trials = input_data.shape
        output_data = []

        if freq_bands != 0:
            # define FIR filter
            fir_filt = firwin(f_order, freq_bands/(srate/2))

        str_hist = ''

        for trial in range(num_of_trials):
            temp_data1 = input_data[:, :, trial]

            # if reref == 0 no re-referencing
            if reref != 0:
                reref_max = int(abs(reref_t) * srate)
                ref_mean = np.mean(temp_data1[:, :reref_max], axis=1)
                temp_data1 = temp_data1 - np.tile(ref_mean, (1, temp_data1.shape[1]))

            # select pre-defined interval
            temp_data1 = temp_data1[:, start_s:end_s]

            print(temp_data1.shape)
            seq_length, tmp2 = temp_data1.shape

            # conduct filtering after interval selection
            # if freq_bands == 0 no filtering
            if freq_bands != 0:
                temp_data1 = filtfilt(fir_filt, [1.0], temp_data1.T).T

            # if p == q no downsampling, if p > q then error
            if p > q:
                print('Error while downsampling, no downsampling performed')
            elif p < q:
                temp_data2 = np.zeros((num_of_channels, seq_length))
                for i in range(num_of_channels):
                    temp_data2[i, :] = resample([0] + list(temp_data1[i, 1:]), int(p), int(q))
            else:
                temp_data2 = temp_data1

            output_data.append(temp_data2)

        if freq_bands != 0:
            str_hist += ' filtered with filter ' + str(freq_bands)
        if reref != 0:
            str_hist += ' re-reference with reference from ' + str(xmin) + ' to ' + str(xmin + reref_t)
        if p < q:
            str_hist += ' down sampled to freq ' + str(srate * p / q)

        # output_struct = None
        # output_struct['data'] = np.array(output_data)
        # output_struct['srate'] = srate * p / q
        # output_struct['xmin'] = start_t
        # output_struct['xmax'] = end_t
        # output_struct['times'] = input_struct['times'][start_s:end_s]
        # output_struct['history'] = input_struct['history'] + str_hist

        return np.array(output_data)


In [104]:
#| export
def erp_feature(input_mat, win_size):
    n_ch, n_T, n_trials = input_mat.shape
    n_dim = int(np.floor(n_T/win_size))
    output_mat = np.zeros((n_ch, n_dim, n_trials))

    for my_trial in range(n_trials):
        my_mat = input_mat[:, :, my_trial]

        for my_dim in range(n_dim):
            my_time = np.arange((my_dim-1)*win_size+1, win_size*my_dim+1)
            output_mat[:, my_dim, my_trial] = np.mean(my_mat[:, my_time], axis=1)

    return output_mat

In [105]:
fp = "../../../EEG-Local/Memory2018/Exp1_SOSI/ALLDATA_1.mat"
loader = DataLoader(fp)

In [107]:
output_data = loader.preprocess_by_trial(0)

(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(432, 1)
(

In [109]:
output_data.shape

(128, 432, 1)

In [ ]:
loader.source_info